In [232]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import mysql.connector

In [233]:
df_attack_meteo = pd.read_csv('../../datos/attacks_meteo.csv', index_col=0)

In [234]:
df_attack_meteo.head(2)

,case_number,year,mes,country,type,age,sex,species,fatal,rh_950mb,...,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,fecha_descarga
0,1791.00.00,1976,Unknown,australia,Unprovoked,27.28,F,UNKNOWN,y,12.6875,...,2.40625,10.265625,1022.328125,2.59375,0.0,1.642202e+151,3.09375,-24.776109,134.755,2022-09-18
1,1788.05.10,1976,May,australia,Boat,27.28,M,UNKNOWN,n,12.6875,...,2.40625,10.265625,1022.328125,2.59375,0.0,1.642202e+151,3.09375,-24.776109,134.755,2022-09-18


In [235]:
class Cargar:
    
    def __init__(self, nombre_bbdd, contraseña):
        
        self.nombre_bbdd = nombre_bbdd
        self.contraseña = contraseña

    # método para crear la BBDD 
    def crear_bbdd(self):

        mydb = mysql.connector.connect(host="127.0.0.1",
                                       user="root",
                                       password=f'{self.contraseña}') 
        mycursor = mydb.cursor()
        print("Conexión realizada con éxito")

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.nombre_bbdd};")
            print(mycursor)
            
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg) 
                
    # método para crear tablas  e insertar datos en ellas   
    def crear_insertar_tabla(self, query):
        
        mydb = mysql.connector.connect(host="127.0.0.1",
                                       user="root",
                                       password=f'{self.contraseña}', 
                                       database=f"{self.nombre_bbdd}") 
        mycursor = mydb.cursor()
        
        try:
            mycursor.execute(query)
            mydb.commit()
          
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)
            

    # método para comprobar si el pais que estamos intentando meter ya existe en la BBDD. 
    # En caso de que ya exista no será necesario que la insertemos de nuevo


In [236]:
bbdd = 'tiburones'
contraseña = 'AlumnaAdalab'

In [237]:
cargar_tiburones = Cargar(bbdd, contraseña)

In [238]:
cargar_tiburones

In [239]:
cargar_tiburones.crear_bbdd()

Conexión realizada con éxito
CMySQLCursor: CREATE DATABASE IF NOT EXISTS tiburones;


In [240]:
tabla_ataques = ''' CREATE TABLE IF NOT EXISTS `ataques` (
    `id_ataques` INT NOT NULL AUTO_INCREMENT, 
    `case_number` VARCHAR(50) NOT NULL,
    `year` INT NOT NULL, 
    `mes` VARCHAR(20) NOT NULL,
    `type` VARCHAR(50) NOT NULL, 
    `country` VARCHAR(50) NOT NULL,  
    `age` FLOAT NOT NULL,
    `species` VARCHAR(50)  NOT NULL,
    `fatal` VARCHAR (20) NOT NULL,
    `sex` VARCHAR (20) NOT NULL,
    PRIMARY KEY (`id_ataques`))
    
    ENGINE = InnoDB;
    '''

In [241]:
tabla_clima = ''' CREATE TABLE IF NOT EXISTS `clima` (
    `id_clima` INT NOT NULL AUTO_INCREMENT, 
    `country` VARCHAR (50) NOT NULL,
    `rh_950mb` FLOAT NOT NULL, 
    `rh_900mb` FLOAT NOT NULL, 
    `rh_850mb` FLOAT NOT NULL, 
    `rh_800mb` FLOAT NOT NULL, 
    `rh_750mb` FLOAT NOT NULL, 
    `rh_700mb` FLOAT NOT NULL,
    `rh_650mb` FLOAT NOT NULL, 
    `rh_600mb` FLOAT NOT NULL, 
    `rh_550mb` FLOAT NOT NULL, 
    `rh_500mb` FLOAT NOT NULL, 
    `rh_450mb` FLOAT NOT NULL, 
    `rh_400mb` FLOAT NOT NULL,
    `rh_350mb` FLOAT NOT NULL, 
    `rh_300mb` FLOAT NOT NULL, 
    `rh_250mb` FLOAT NOT NULL, 
    `rh_200mb` FLOAT NOT NULL, 
    `windspeed_950mb` FLOAT NOT NULL,
    `windspeed_900mb` FLOAT NOT NULL, 
    `windspeed_850mb` FLOAT NOT NULL, 
    `windspeed_800mb` FLOAT NOT NULL,
    `windspeed_750mb` FLOAT NOT NULL, 
    `windspeed_700mb` FLOAT NOT NULL, 
    `windspeed_650mb` FLOAT NOT NULL,
    `windspeed_600mb` FLOAT NOT NULL, 
    `windspeed_550mb` FLOAT NOT NULL, 
    `windspeed_500mb` FLOAT NOT NULL,
    `windspeed_450mb` FLOAT NOT NULL, 
    `windspeed_400mb` FLOAT NOT NULL, 
    `windspeed_350mb` FLOAT NOT NULL,
    `windspeed_300mb` FLOAT NOT NULL, 
    `windspeed_250mb` FLOAT NOT NULL, 
    `windspeed_200mb` FLOAT NOT NULL, 
    `timepoint` FLOAT NOT NULL,
    `cloudcover` FLOAT NOT NULL, 
    `highcloud` FLOAT NOT NULL, 
    `midcloud` FLOAT NOT NULL, 
    `lowcloud` FLOAT NOT NULL, 
    `temp2m` FLOAT NOT NULL, 
    `lifted_index` FLOAT NOT NULL, 
    `rh2m` FLOAT NOT NULL, 
    `msl_pressure` FLOAT NOT NULL,
    `prec_amount` FLOAT NOT NULL, 
    `snow_depth` FLOAT NOT NULL, 
    `wind10m.direction` REAL DEFAULT NULL,
    `wind10m.speed` FLOAT DEFAULT NULL, 
    `latitud` FLOAT NOT NULL, 
    `longitud` FLOAT NOT NULL, 
    `fecha_descarga` VARCHAR (20) NOT NULL,
    PRIMARY KEY (`id_clima`))
    
    ENGINE = InnoDB;
    '''

Creamos tablas de nuestra bbdd 'tiburones'

In [242]:
cargar_tiburones.crear_insertar_tabla(tabla_ataques)

In [243]:
cargar_tiburones.crear_insertar_tabla(tabla_clima)

Insertamos datos en nuestras tablas

In [244]:
# Primero filtramos nuestro df para que nos de las querys solo de las columnas 
# que queremos agregar en nuestra tabla
df_attacks = df_attack_meteo[['case_number',
 'year',
 'mes',
 'country',
 'type',
 'age',
 'sex',
 'species',
 'fatal']]

In [245]:
df_attacks.head(1)

,case_number,year,mes,country,type,age,sex,species,fatal
0,1791.00.00,1976,Unknown,australia,Unprovoked,27.28,F,UNKNOWN,y


In [246]:
# tabla ataques: creamos la query que nos inserta datos en la tabla ataques

for indice, fila in df_attacks.iterrows():
    

    query_ataques = f"""
                INSERT INTO ataques (case_number, year, mes, type, country, age, species,
                fatal, sex)
                VALUES ("{fila['case_number']}", "{fila["year"]}", "{fila["mes"]}", "{fila["type"]}", "{fila["country"]}",
                "{fila["age"]}", "{fila["species"]}", "{fila["fatal"]}", "{fila["sex"]}");
                """
    
    cargar_tiburones.crear_insertar_tabla(query_ataques)




Para insertar los datos en la tabla clima podríamos filtrarlos a partir de nuestra tabla attacks_meteo donde ya tenemos toda la información recopilada. Sin embargo, esto incluiría agregar algunos pasos ya que nos agregaría todos los duplicados que tenemos. Por lo tanto, hemos decido insertar la información a partir de la tabla creada anteriormente llamada meteo_medias, donde tenemos solo las medias para cada país y fecha. 

In [248]:
df2 = pd.read_csv('../../datos/meteo_medias.csv', index_col=0)

In [249]:
df2.shape

(5, 49)

In [250]:
df2.head(1)

,pais,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,...,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,fecha_descarga
0,australia,12.6875,12.15625,9.625,5.40625,2.65625,1.6875,1.1875,0.6875,0.625,...,2.40625,10.265625,1022.328125,2.59375,0.0,1.642202e+151,3.09375,-24.776109,134.755,2022-09-18


In [252]:
# tabla clima: creamos la query que nos inserte los datos en la tabla clima


for indice, fila in df2.iterrows():

    query_clima = f"""
                INSERT INTO clima (country, rh_950mb, rh_900mb,
                                    rh_850mb,
                                    rh_800mb,
                                    rh_750mb,
                                    rh_700mb,
                                    rh_650mb,
                                    rh_600mb,
                                    rh_550mb,
                                    rh_500mb,
                                    rh_450mb,
                                    rh_400mb,
                                    rh_350mb,
                                    rh_300mb,
                                    rh_250mb,
                                    rh_200mb,
                                    windspeed_950mb,
                                    windspeed_900mb,
                                    windspeed_850mb,
                                    windspeed_800mb,
                                    windspeed_750mb,
                                    windspeed_700mb,
                                    windspeed_650mb,
                                    windspeed_600mb,
                                    windspeed_550mb,
                                    windspeed_500mb,
                                    windspeed_450mb,
                                    windspeed_400mb,
                                    windspeed_350mb,
                                    windspeed_300mb,
                                    windspeed_250mb,
                                    windspeed_200mb,
                                    timepoint,
                                    cloudcover,
                                    highcloud,
                                    midcloud,
                                    lowcloud,
                                    temp2m,
                                    lifted_index,
                                    rh2m,
                                    msl_pressure,
                                    prec_amount,
                                    snow_depth,
                                    latitud,
                                    longitud,
                                    fecha_descarga) 
                VALUES ("{fila["pais"]}","{fila["rh_950mb"]}", "{fila["rh_900mb"]}",
                                    "{fila["rh_850mb"]}",
                                    "{fila["rh_800mb"]}",
                                    "{fila["rh_750mb"]}",
                                    "{fila["rh_700mb"]}",
                                    "{fila["rh_650mb"]}",
                                    "{fila["rh_600mb"]}",
                                    "{fila["rh_550mb"]}",
                                    "{fila["rh_500mb"]}",
                                    "{fila["rh_450mb"]}",
                                    "{fila["rh_400mb"]}",
                                    "{fila["rh_350mb"]}",
                                    "{fila["rh_300mb"]}",
                                    "{fila["rh_250mb"]}",
                                    "{fila["rh_200mb"]}",
                                    "{fila["windspeed_950mb"]}",
                                    "{fila["windspeed_900mb"]}",
                                    "{fila["windspeed_850mb"]}",
                                    "{fila["windspeed_800mb"]}",
                                    "{fila["windspeed_750mb"]}",
                                    "{fila["windspeed_700mb"]}",
                                    "{fila["windspeed_650mb"]}",
                                    "{fila["windspeed_600mb"]}",
                                    "{fila["windspeed_550mb"]}",
                                    "{fila["windspeed_500mb"]}",
                                    "{fila["windspeed_450mb"]}",
                                    "{fila["windspeed_400mb"]}",
                                    "{fila["windspeed_350mb"]}",
                                    "{fila["windspeed_300mb"]}",
                                    "{fila["windspeed_250mb"]}",
                                    "{fila["windspeed_200mb"]}",
                                    "{fila["timepoint"]}",
                                    "{fila["cloudcover"]}",
                                    "{fila["highcloud"]}",
                                    "{fila["midcloud"]}",
                                    "{fila["lowcloud"]}",
                                    "{fila["temp2m"]}",
                                    "{fila["lifted_index"]}",
                                    "{fila["rh2m"]}",
                                    "{fila["msl_pressure"]}",
                                    "{fila["prec_amount"]}",
                                    "{fila["snow_depth"]}",
                                    "{fila["latitud"]}",
                                    "{fila["longitud"]}",
                                    "{fila['fecha_descarga']}"
                                    ) ;
     
               """

    cargar_tiburones.crear_insertar_tabla(query_clima)    

Hemos logrado crear la base de datos, las tablas ataques y clima, y también hemos insertado los datos. A futuro nos gustaría agregar una tabla que sea países, donde incluiríamos los países con su latitud y longitud. De esta manera, podríamos eliminar redundancias en nuestra base de datos, como por ejemplo que la columna 'country' esté repetida tanto en ataques como clima. Haciendo este cambio, id_pais podría funcionar como foreign key en ambas tablas. 